In [10]:
import os
from numpy import geomspace
from PyFoam.RunDictionary.SolutionDirectory import SolutionDirectory
from PyFoam.RunDictionary.ParsedParameterFile import ParsedParameterFile
from PyFoam.Basics.DataStructures import Vector
from colorcet import cm

In [3]:
viscosity_vals = geomspace(0.00001, 0.01, 200)  # Varying viscosity values
reynolds_no = 1. * 0.1 / viscosity_vals 

### Create Case Directories

In [4]:
# Genererate cases with different viscosities
for i, Re in enumerate(reynolds_no):
    # === Laminar case ===
    if Re < 2000:
        templateDir = SolutionDirectory("poiseuilleFlowLaminarTemplate", archive=None, paraviewLink=False)
    # === Turbulent case ===
    else:
        templateDir = SolutionDirectory("poiseuilleFlowTurbulentTemplate", archive=None, paraviewLink=False)
    # Create a new case
    case = templateDir.cloneCase(f"poiseuilleFlowCases/poiseuilleFlow-Re{Re:.2f}", paraviewLink=False)
    # Get the directory with viscosity parameter
    transportProps = ParsedParameterFile(os.path.join(case.name, "constant", "transportProperties"))
    # Set appropriate value of viscosity
    transportProps["nu"] = viscosity_vals[i]
    # Save the transport properties dictionary
    transportProps.writeFile()

### Export graphs for ML

In [11]:
import pyvista as pv

In [13]:
# Generate pressure and velocity contours and pathlines for each case in poiseuilleFlow dir
case_dir_list = os.listdir("poiseuilleFlowCases/")
for case in case_dir_list:
    # Define a path to a folder
    case_path = os.path.join("poiseuilleFlowCases/", case)
    # Get the path to VTK folder
    vtk_path = os.path.join(case_path, "VTK")
    # Get the list of appropriate files
    vtk_files = os.listdir(vtk_path)
    # Get only the files with .vtm extension
    vtk_files = [a_file for a_file in vtk_files if ".vtm" in a_file and ".series" not in a_file]
    # Get the last iteration
    vtk_file = max(vtk_files, key=lambda f: int(f.split("_")[-1].split(".")[0]))
    # Create path to the VTK file
    vtk_file_path = os.path.join(vtk_path, vtk_file)
    # Get the Reynolds No.
    Re_num = vtk_file.split("-")[-1].split("_")[0]
    # Laminar case - we don't have to do mirror reflection
    if float(Re_num[2:]) < 2000:
        # Define appropriate camera position
        camera_position = [(0.5, 0.05, 1.25), (0.5, 0.05, 0), (0, 1, 0)]
        mesh = pv.read(vtk_file_path)
        vol = mesh[0]
        # Streamline plot
        combined_mesh = mesh.combine()
        combined_mesh.set_active_vectors("U")
        one_streamline = combined_mesh.streamlines(
            pointa=(0.5, 0, 0),
            pointb=(0.5, 0.1, 0),
            n_points=15,
            max_length=100.0,
            compute_vorticity=False,  # vorticity already exists in dataset
        )
    else:
        # Define appropriate camera position
        camera_position = [(0.5, 0.0, 1.25), (0.5, 0.0, 0), (0, 1, 0)]
        mesh = pv.read(vtk_file_path)
        mesh = mesh.combine()
        # Reflect along x
        reflected_mesh = mesh.reflect(normal=(0, 1, 0))
        # Merge original and mirrored
        vol = mesh.merge(reflected_mesh)
        # Streamline plot
        vol.set_active_vectors("U")
        one_streamline = vol.streamlines(
            pointa=(0.5, -0.05, 0),
            pointb=(0.5, 0.05, 0),
            n_points=15,
            max_length=100.0,
            compute_vorticity=False,  # vorticity already exists in dataset
        )    
    # Create a plotter
    plotter = pv.Plotter(off_screen=True)
    windows_size = (500, 300)
    # Pressure field plot
    plotter.add_mesh(vol, scalars="p", cmap='linear_grey_0_100_c0', show_scalar_bar=False)
    plotter.camera_position = camera_position
    plotter.window_size = windows_size
    plotter.screenshot(filename=f"mlDataset/poiseuilleFlow/images/p_field{Re_num}.png")
    plotter.clear()
    # Velocity field plot
    plotter.add_mesh(vol, scalars="U", cmap="linear_grey_0_100_c0", show_scalar_bar=False)
    plotter.screenshot(filename=f"mlDataset/poiseuilleFlow/images/U_field{Re_num}.png")
    plotter.clear()
    # Streamline plot
    plotter.add_mesh(one_streamline, color="black")
    plotter.screenshot(filename=f"mlDataset/poiseuilleFlow/images/streamlines{Re_num}.png")
    plotter.close()

### Create a mask for poiseuille Flow

In [5]:
plotter = pv.Plotter()
camera_position = [(0.5, 0.05, 1.25), (0.5, 0.05, 0), (0, 1, 0)]
mesh = pv.read("poiseuilleFlowCases/poiseuilleFlow-Re11.10/VTK/poiseuilleFlow-Re11.10_79.vtm")
plotter.add_mesh(mesh[0], color="white", show_edges=False)
plotter.camera_position = camera_position
plotter.background_color = 'black'
plotter.window_size = (500, 300)
plotter.screenshot(filename=f"mlDataset/poiseuilleFlow/mask/poiseuilleFlowMask.png")

pyvista_ndarray([[[0, 0, 0],
                  [0, 0, 0],
                  [0, 0, 0],
                  ...,
                  [0, 0, 0],
                  [0, 0, 0],
                  [0, 0, 0]],

                 [[0, 0, 0],
                  [0, 0, 0],
                  [0, 0, 0],
                  ...,
                  [0, 0, 0],
                  [0, 0, 0],
                  [0, 0, 0]],

                 [[0, 0, 0],
                  [0, 0, 0],
                  [0, 0, 0],
                  ...,
                  [0, 0, 0],
                  [0, 0, 0],
                  [0, 0, 0]],

                 ...,

                 [[0, 0, 0],
                  [0, 0, 0],
                  [0, 0, 0],
                  ...,
                  [0, 0, 0],
                  [0, 0, 0],
                  [0, 0, 0]],

                 [[0, 0, 0],
                  [0, 0, 0],
                  [0, 0, 0],
                  ...,
                  [0, 0, 0],
                  [0, 0, 0],
             